In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import glob
import cv2
import numpy as np
import PIL


/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [5]:
datagen = ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            rescale=1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

In [6]:
img = load_img('/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/eyespot/Apogon_Apogon cavitiensis_-1692469642.jpg')

In [7]:
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [12]:
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview', save_prefix='eyespot', save_format='jpeg'):
    i += 1
    if i>20:
        break

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/'
validation_data_dir = '/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species_validation/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 2
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

train_generator[1][0][0].shape

Found 70 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


(150, 150, 3)

In [4]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')
model.save('second_try.h5')

Epoch 1/2
125/125 [==============================] - 112s 898ms/step - loss: 0.7092 - acc: 0.6025 - val_loss: 1.0261 - val_acc: 0.3000
Epoch 2/2
125/125 [==============================] - 109s 873ms/step - loss: 0.4438 - acc: 0.8010 - val_loss: 3.2146 - val_acc: 0.2000


## Reload model and use it to predict

In [18]:
import cv2

def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    classification, _, species = filepath.partition('_')
    
    #res = model.predict(img)
    return(img, classification, species)

#run through all images, classify,
#and write out the classification category, 
#the confidence/weight of the prediction, 
#the species name, and the family. 

def predict(model, img):
    prediction_weights = model.predict(img[0])
    prediction = model.predict_classes(img[0])
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"
    fam = img[2]
    correct_class = img[1]
    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)


In [9]:
from keras.models import load_model

mod = load_model("second_try.h5")

eyepath = "eyespot_Abudefduf_Abudefduf sparoides_929157376.jpg"
noeyepath = "noeyespot_Abudefduf_Abudefduf bengalensis_880932306.jpg"

predict(mod, getFish(eyepath))

Model classification: eyespot 
Correct classification: eyespot 
Prediction wieghts: [[4.43086058e-01 1.03738705e-04]] 
Type: Abudefduf_Abudefduf sparoides_929157376.jpg 



('eyespot',
 'eyespot',
 array([[4.43086058e-01, 1.03738705e-04]], dtype=float32),
 'Abudefduf_Abudefduf sparoides_929157376.jpg')

In [13]:
import glob

images = glob.glob('*.jpg')
len(images)

80

In [35]:
import csv

with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Correct Classification', 'Prediction Weights', 'Type'])

    true_eye=0
    true_noeye=0
    false_eye=0
    false_noeye=0

    for i in range(len(images)):
        info = predict(mod, getFish(images[i]))
        filewriter.writerow([info[0], info[1], info[2], info[3]])

        #computing info for confusion matrix
        if (info[0] == info[1]):
            if (info[0] == 'eyespot'):
                true_eye += 1
            else:
                true_noeye += 1
        else:
            if (info[0] == 'eyespot'):
                false_eye += 1
            else:
                false_noeye += 1

In [36]:
import pandas as pd

data = {'Eyespots':[true_eye, false_noeye], 'No Eyespots':[false_eye, true_noeye] }
confusion_matrix = pd.DataFrame(data = data)
confusion_matrix = confusion_matrix.rename({0:'Eyespots (Model Output)', 1:'No Eyespots (Model Output)'},axis='index')
confusion_matrix

,Eyespots,No Eyespots
Eyespots (Model Output),37,13
No Eyespots (Model Output),3,27
